In [ ]:
!pip install dotenv
!pip install streamlit
!npm install localtunnel
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 2s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [ ]:
!pip  install streamlit

In [ ]:
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
%%writefile CodeModel.py

import os

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from accelerate import init_empty_weights, infer_auto_device_map, load_checkpoint_in_model
import torch
import time
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()
token = os.getenv("HF_TOKEN")
login(token)


def generate_code(prompt):
    model_id = "codellama/CodeLlama-7b-hf"
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Load model with disk offloading
    with init_empty_weights():
        model = AutoModelForCausalLM.from_pretrained(model_id)
    device_map = infer_auto_device_map(model, no_split_module_classes=["LlamaDecoderLayer"],
                                       max_memory={"cpu": "48GiB"})
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map=device_map,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True
    )

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map=device_map,
        torch_dtype=torch.float16
    )

    start = time.time()
    sequences = pipe(
        prompt,
        do_sample=True,
        top_k=10,
        temperature=0.1,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=128
    )
    result = ''.join([s['generated_text'] for s in sequences])
    print(result)
    end = time.time()
    return result, start, end

Overwriting CodeModel.py


In [ ]:
%%writefile app.py

import streamlit as st
import time
from CodeModel import generate_code
st.header("CodeGenie: AI-powered code generator")

with st.form("my_form"):
    user_input = st.text_area("Enter your text prompt below and click the button to submit.")
    submit = st.form_submit_button(label="Submit text prompt")

if submit:
    with st.spinner(text="Generating code... It may take some time"):
        code, start, end = generate_code(prompt=user_input)
        hours, rem = divmod(end - start, 3600)
        minutes, seconds = divmod(rem, 60)

        st.success(
            "Processing time: {:0>2}:{:0>2}:{:05.2f}.".format(
                int(hours), int(minutes), seconds
            )
        )
        st.code(code, language='python')

    st.sidebar.markdown("## Guide")
    st.sidebar.info("This tool uses salesforce codegen 350m parameters")



Overwriting app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.85.242.49


**Copy that number and paste it on the website as password on next functional cell**

**Click "your url is:" link and paste that number**

In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.85.242.49:8501

your url is: https://shiny-trains-open.loca.lt
2025-06-13 07:06:51.521131: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749798411.552187    6389 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749798411.560815    6389 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-13 07:06:51.590453: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following ins